# MODELO LSTM

In [ ]:
import pandas as pd
import numpy as np
np.bool = np.bool_
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import RMSprop, Adam
import tensorflow as tf

### Procesamiento de Datos

Lectura de los archivos y agrupamiento de ventas por período y producto.

In [ ]:
sells = pd.read_csv("sell-in.txt", sep = "\t")
filter_id = pd.read_csv("productos_a_predecir.txt", sep = "\t")
sells = sells[sells.product_id.isin(filter_id.product_id)]
productos = sells.product_id[sells.periodo == max(sells.periodo)]
productos = np.unique(productos)
df_sells = sells.groupby(["product_id", "periodo"])["tn"].aggregate('sum').reset_index()
df_sells.sort_values(["product_id", "periodo"], inplace = True)
descripcion = pd.read_csv(r"C:\Users\rodri\OneDrive\Documentos\Maestria Ciencia de Datos\Labo 3\DataSets\tb_productos.txt", sep = "\t")
descripcion = descripcion[descripcion.product_id.isin(filter_id.product_id)]

In [ ]:
df_sells.head()

Se crea un dataset con todos los periodos para todos los productos y se realiza un join con el dataset leido.

In [ ]:
df_completo = pd.DataFrame(np.array(np.meshgrid(np.unique(df_sells.product_id), 
                                                np.unique(df_sells.periodo))).T.reshape(-1,2),
                           columns = ["product_id", "periodo"])

df_completo = df_completo.merge(df_sells, how="left", on=["product_id","periodo"])

Se transforma el dataset de formato long a wide.

In [ ]:
df_lstm = df_completo.pivot(columns="product_id", values = "tn", index = "periodo")
df_lstm.columns.name = None

Se utiliza la función interpole para identificar los meses en los que no hubo ventas de productos existentes.

In [ ]:
interpolado = df_lstm.interpolate(axis=0,limit_area="inside",limit_direction="both")
df_lstm.fillna(-1, inplace=True)
df_lstm[(interpolado>0) & (df_lstm == -1)] = 0

Se crea un dataset indicando la antigüedad de cada producto.

In [ ]:
lista_antiguedad = []
nombres_antiguedad = []
for k in df_lstm.columns:
  df_antiguedad = df_lstm.loc[:,k]
  antiguedad = []
  nombre = "antiguedad_" + str(k)
  if sum(df_antiguedad == -1) > 0:
    antiguedad = [-1] * sum(df_antiguedad == -1)
    antiguedad.extend(range(36 - sum(df_antiguedad == -1)))
  if (np.mean(df_antiguedad[24:]) < 0.5 * np.mean(df_antiguedad[12:24])) & (len(antiguedad) == 0):
    antiguedad = [-2] * 36
  if len(antiguedad) == 0:
    antiguedad = [-3] * 36
  
  lista_antiguedad.append(antiguedad)
  nombres_antiguedad.append(nombre)

diccionario = dict(zip(nombres_antiguedad, lista_antiguedad))

df_con_antiguedad = pd.DataFrame(diccionario)
df_con_antiguedad.index = lstm_delta.index


Se vuelven a imputar los meses de los producto que no existían en NAN.

In [ ]:
df_lstm[df_lstm == -1] = np.NaN

#### Escalado de datos

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(df_lstm)
df_lstm_scale = pd.DataFrame(scaler.transform(df_lstm), columns=df_lstm.columns, index = df_lstm.index)

#### Imputación de NA

Se utiliza el promedio de ventas para cada mes para imputar los NA de productos que no existían.

In [ ]:
for i in np.unique(descripcion["cat1"]) :
  for j in np.unique(descripcion["cat2"]) :
    ids = list(np.unique(descripcion[(descripcion["cat1"] == i) & (descripcion["cat2"] == j)].product_id))
    promedios = pd.DataFrame(np.tile(df_lstm_scale[ids].mean(axis = 1), (len(ids), 1)).transpose(), columns = ids, index=df_lstm_scale.index)
    df_lstm_scale[ids] = df_lstm_scale[ids].fillna(promedios)

#### Imputación de ventas en 201908

Se imputan las ventas de este mes con un interpolación.

In [ ]:
df_lstm_scale.iloc[df_lstm_scale.index == 201908,:] = np.NaN
df_lstm_scale.interpolate(method='polynomial',order = 2, axis=0, inplace=True)

#### Creación dataframe de ventas escalado

In [ ]:
df_lstm_2 = pd.DataFrame(scaler.inverse_transform(df_lstm_scale), columns=df_lstm.columns, index = df_lstm.index)

#### Calculo de pesos a incorporar al modelo

In [ ]:
pesos = list(df_lstm_2.loc[[201901,201902,201903],:].mean(axis = 0))

### Creación de DataFrame con variables delta lag

In [ ]:
columnas = df_lstm.columns

Se crean 12 variables para cada producto por la diferencia entre las ventas de un periodo con el del mes anterior al año anterior.

In [ ]:
def create_lagged_features(data, lag=12):
    list_df = []
    for i in range(1, lag + 1):
        data_lag = data.diff(periods=i, axis=0)
        data_lag.columns = f"delta_{i}_" + columnas.astype(str)
        data_lag.iloc[0:i,:] = data_lag.iloc[12:i+12,:]
        list_df.append(data_lag)
    return pd.concat(list_df, axis=1)

In [ ]:
lstm_delta = create_lagged_features(df_lstm_scale, lag=12)

### Creación de variable roll mean

Se crean variables de roll mean con el objetivo de identificar si se compró mucho o poco en los últimos meses.

In [ ]:
def create_mean_features(data, mean):
    list_df = []
    for i in range(1, mean - 1):
        data_mean = data.rolling(i+1, min_periods=0).sum()
        data_mean.columns = f"mean_{i}_" + columnas.astype(str)
        list_df.append(data_mean)
    return pd.concat(list_df, axis=1)

In [ ]:
lstm_mean = create_mean_features(df_lstm_scale, 6)

### Concatenación de los DataFrames

In [ ]:
df_modelo = pd.concat([df_lstm_scale,lstm_delta,lstm_mean, df_con_antiguedad], axis=1)

## Entrenamiento y predicción del modelo

Se crearán ventas de 24 meses para el entrenamiento y se utilizarán 2 períodos de validación y 1 de testeo.

In [ ]:
X_train = np.array(df_modelo[0:32])
X_validate = np.array(df_modelo[8:33])
X_test = np.array(df_modelo[10:34])

y_train = np.array(df_modelo[1:33])
y_validate = np.array(df_modelo[10:35])
y_test = np.array(df_modelo[12:])

Se les da el formato correspondiente a los datos para que sean introducidos al LSTM. Se crea una función para los datos X y otra para los datos Y.

In [ ]:
def crear_dataset_supervisado(array, input_length, output_length):

    # Inicialización
    X = [] # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado <-- <--- ¡esta parte de la función se ejecuta en este caso!
        fils, cols = array.shape

    # Generar los arreglos
    for i in range(fils-input_length+1):
        X.append(array[i:i+INPUT_LENGTH,0:cols])

    
    # Convertir listas a arreglos de NumPy
    X = np.array(X)

    
    return X

In [ ]:
def crear_dataset_supervisado_2(array, input_length, output_length):

    # Inicialización
    X = [] # Listados que contendrán los datos de entrada y salida del modelo
    shape = array.shape
    if len(shape)==1: # Si tenemos sólo una serie (univariado)
        fils, cols = array.shape[0], 1
        array = array.reshape(fils,cols)
    else: # Multivariado <-- <--- ¡esta parte de la función se ejecuta en este caso!
        fils, cols = array.shape

    # Generar los arreglos
    for i in range(fils-input_length+1):
        X.append(array[fils - i - output_length:fils - i,0:cols])

    
    # Convertir listas a arreglos de NumPy
    X = np.array(X)

    
    return X

Se utillizan las categorías cat1 y cat2 para agrupar los datos y pasarlos por el modelo. Por lo tanto, se debe realizar un loop y preparar los datos de entrenamiento y testeo para cada loop. Luego se concatenan las predicciones en un DataFrame.

In [ ]:
# Ajustar parámetros para reproducibilidad del entrenamiento
tf.random.set_seed(123)
tf.config.experimental.enable_op_determinism()

predicciones = pd.DataFrame(columns=["product_id", "tn"])
for i in np.unique(descripcion["cat1"]) :
  for j in np.unique(descripcion["cat2"]) :
    ids = list(np.unique(descripcion[(descripcion["cat1"] == i) & (descripcion["cat2"] == j)].product_id))
    if len(ids) > 0 :
        df = df_lstm_scale[ids]
        lstm_delta = create_lagged_features(df_lstm_scale, lag=12)
        lstm_mean = create_mean_features(df_lstm_scale, 6)
        df_modelo = pd.concat([df,lstm_delta,lstm_mean], axis=1)
        df_final = df_lstm_2[ids]

        X_train = np.array(df_modelo[0:32])
        X_validate = np.array(df_modelo[8:33])
        X_test = np.array(df_modelo[10:34])

        y_train = np.array(df[1:33])
        y_validate = np.array(df[10:35])
        y_test = np.array(df[12:])
        # Crear los datasets de entrenamiento, prueba y validación y verificar sus tamaños
        INPUT_LENGTH = 24    # Hiperparámetro
        OUTPUT_LENGTH = 1    # Modelo multi-step

        x_tr = crear_dataset_supervisado(X_train, INPUT_LENGTH, OUTPUT_LENGTH)
        x_vl = crear_dataset_supervisado(X_validate, INPUT_LENGTH, OUTPUT_LENGTH)
        x_ts = crear_dataset_supervisado(X_test, INPUT_LENGTH, OUTPUT_LENGTH)

        y_tr = crear_dataset_supervisado_2(y_train, INPUT_LENGTH, OUTPUT_LENGTH)
        y_vl = crear_dataset_supervisado_2(y_validate, INPUT_LENGTH, OUTPUT_LENGTH)
        y_ts = crear_dataset_supervisado_2(y_test, INPUT_LENGTH, OUTPUT_LENGTH)



        # El modelo
        N_UNITS = 500 # Tamaño del estado oculto (h) y de la celdad de memoria (c) (128)
        INPUT_SHAPE = (x_tr.shape[1], x_tr.shape[2]) # 24 (horas) x 13 (features)

        modelo = Sequential()
        modelo.add(LSTM(N_UNITS, input_shape=INPUT_SHAPE, return_sequences=True,dropout=0.2))
        #modelo.add(LSTM(N_UNITS, return_sequences=True))
        modelo.add(LSTM(N_UNITS, return_sequences=True,dropout=0.2))
        modelo.add(LSTM(N_UNITS, return_sequences=True,dropout=0.2))
        modelo.add(LSTM(N_UNITS))
        # Y lo único que cambia con respecto al modelo multivariado + multi-step es
        # el tamaño deldato de salida (4 horas)
        modelo.add(Dense(df.shape[1], activation='linear')) # activation = 'linear' pues queremos pronosticar (regresión)

        # Pérdida: se usará el RMSE (root mean squared error) para el entrenamiento
        # pues permite tener errores en las mismas unidades de la temperatura
        def root_mean_squared_error(y_true, y_pred):
            rmse = tf.math.sqrt(tf.math.reduce_mean(tf.square(y_pred-y_true)))
            return rmse

        pesos_filtro = df_lstm_2[ids]
        pesos = list(pesos_filtro.loc[[201901,201902,201903],:].mean(axis = 0))
        # Compilación
        optimizador = RMSprop(learning_rate=5e-4) # 5e-5
        modelo.compile(
            optimizer = optimizador,
            loss = "MeanSquaredError",
            loss_weights = pesos
        )   

        EPOCHS = 100 # Hiperparámetro
        BATCH_SIZE = 5 # Hiperparámetro
        historia = modelo.fit(
            x = x_tr,
            y = y_tr,
            batch_size = BATCH_SIZE,
            epochs = EPOCHS,
            validation_data = (x_vl, y_vl),
            verbose=2
        )

        scaler_final = StandardScaler()

        scaler_final = scaler_final.fit(df_modelo)
        df_escalas = pd.DataFrame({"product_id" : df_modelo.columns,"Mean" : scaler_final.mean_,"STD" : scaler_final.scale_})

        #X_final = scaler_final.transform(df_lstm[-12:])
        X_final = scaler_final.transform(df_modelo[-24:])

        X_final = crear_dataset_supervisado(X_final, INPUT_LENGTH, OUTPUT_LENGTH)
        # 1. Generar las predicciones sobre el set de prueba
        y_final = modelo.predict(X_final, verbose=0)
        
        scaler_2 = StandardScaler()

        scaler_2 = scaler_2.fit(df_lstm[ids])
        # 2. Realizar la transformación inversa de las predicciones para llevar sus
        # valores a la escala original
        y_final = scaler_2.inverse_transform(y_final)

        df_prediccion = pd.DataFrame({"product_id" : df_final.columns,"tn" : y_final[0,:]})

        predicciones = pd.concat([predicciones, df_prediccion], axis=0)


Se graban las predicciones en un CSV.

In [ ]:
predicciones.to_csv("LSTM_completo_1.csv", index = False)